In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
article_pos_file = "data/B.1_2022-08-22.og.aln.pruned.tree.amino_acid.reconstruction.csv"
article_df = pd.read_csv(article_pos_file, sep=',')
print(len(article_df))
article_df.head()

800


,site,gene,direction,snp,dimer,apobec,aa_position,parent,parent_codon,parent_aa,child,child_codon,child_aa,mutation_category,score,prediction,homoplasy,occurrence
0,3111,OPG003_CDS_176,reverse,G->A,GA,True,2.0,Node1,TCC,S,Node2,TTC,F,nonsynonymous,155.0,radical,False,1
1,39139,OPG057_CDS_142,reverse,C->T,TC,True,3.0,Node1,GCG,A,Node2,GCA,A,synonymous,NaN,NaN,False,1
2,73239,OPG093_CDS_107,forward,G->A,GA,True,2.0,Node1,AGA,R,Node2,AAA,K,nonsynonymous,26.0,conservative,False,1
3,74205,OPG094_CDS_106,forward,G->A,GA,True,1.0,Node1,GAA,E,Node2,AAA,K,nonsynonymous,56.0,moderately conservative,False,1
4,77383,OPG098_CDS_102,forward,G->A,GA,True,2.0,Node1,TGA,*,Node2,TAA,*,synonymous,NaN,NaN,False,1


In [3]:
article_df.columns

Index(['site', 'gene', 'direction', 'snp', 'dimer', 'apobec', 'aa_position',
       'parent', 'parent_codon', 'parent_aa', 'child', 'child_codon',
       'child_aa', 'mutation_category', 'score', 'prediction', 'homoplasy',
       'occurrence'],
      dtype='object')

In [4]:
article_df['REF'] = article_df.apply(lambda row: row['snp'].split('->')[0], axis = 1)
article_df['ALT'] = article_df.apply(lambda row: row['snp'].split('->')[1], axis = 1)
article_df = article_df[['site', 'gene', 'direction', 'REF', "ALT", 'dimer', 'apobec', 'aa_position',
       'parent', 'parent_codon', 'parent_aa', 'child', 'child_codon',
       'child_aa', 'mutation_category', 'score', 'prediction', 'homoplasy',
       'occurrence']]
article_df.head()

,site,gene,direction,REF,ALT,dimer,apobec,aa_position,parent,parent_codon,parent_aa,child,child_codon,child_aa,mutation_category,score,prediction,homoplasy,occurrence
0,3111,OPG003_CDS_176,reverse,G,A,GA,True,2.0,Node1,TCC,S,Node2,TTC,F,nonsynonymous,155.0,radical,False,1
1,39139,OPG057_CDS_142,reverse,C,T,TC,True,3.0,Node1,GCG,A,Node2,GCA,A,synonymous,NaN,NaN,False,1
2,73239,OPG093_CDS_107,forward,G,A,GA,True,2.0,Node1,AGA,R,Node2,AAA,K,nonsynonymous,26.0,conservative,False,1
3,74205,OPG094_CDS_106,forward,G,A,GA,True,1.0,Node1,GAA,E,Node2,AAA,K,nonsynonymous,56.0,moderately conservative,False,1
4,77383,OPG098_CDS_102,forward,G,A,GA,True,2.0,Node1,TGA,*,Node2,TAA,*,synonymous,NaN,NaN,False,1


Searching for amino acids in positions

In [16]:
def load_genome(fasta_file):
    """Load the genome from a FASTA file."""
    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    new_genome = genome["NC_063383.1"].seq
    return new_genome

def load_gtf(gtf_file):
    """Load the GTF file into a pandas DataFrame."""
    gtf_columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
    gtf = pd.read_csv(gtf_file, sep="\t", comment='#', header=None, names=gtf_columns)
    return gtf

def find_parent_mutation(genome, parent, table=article_df):
    parent_filter_df = table[table['child']==parent]
    parent_filter_df = parent_filter_df.reset_index(drop=True)
    new_parent = parent_filter_df.loc[0, "parent"]
    sites_list = parent_filter_df.site.tolist()
    
    if new_parent == "Node1":
        new_genome = list(genome)
        for row in range(0, len(parent_filter_df)):
            position = parent_filter_df.loc[row, "site"]
            alt_nucl = parent_filter_df.loc[row, "ALT"]
            new_genome[position-1] = alt_nucl
        return Seq(''.join(new_genome))
    else:
        new_genome = list(genome)
        for row in range(0, len(parent_filter_df)):
            position = parent_filter_df.loc[row, "site"]
            alt_nucl = parent_filter_df.loc[row, "ALT"]
            new_genome[position-1] = alt_nucl
        return find_parent_mutation(new_genome, new_parent)

    
def get_cds(gtf, chromosome, position):
    """Find the CDS where the mutation is located."""
    cds_rows = gtf[(gtf['seqname'] == chromosome) & 
                   (gtf['feature'] == 'CDS') & 
                   (gtf['start'] <= position) & 
                   (gtf['end'] >= position)]
    return cds_rows

def translate_codon(codon):
    """Translate a codon to its corresponding amino acid."""
    return str(Seq(codon).translate(table=CodonTable.unambiguous_dna_by_id[11]))

def detect_mutation_impact(genome, gtf, chromosome, position, ref, alt):
    """Detect the amino acid change caused by a mutation."""
    cds = get_cds(gtf, chromosome, position)
    
    if cds.empty:
        ref_nucl = genome[position-1]
        return '-', '-', '-', '-', ref_nucl, '-'
    
    strand = cds.iloc[0]['strand']
    gene_start = cds.iloc[0]['start']
    
    # Calculate the codon position
    codon_start = gene_start + ((position - gene_start) // 3) * 3
    
    # Extract the codon
    codon_seq = genome[codon_start-1:codon_start+2]
    if strand == '-':
        codon_seq = codon_seq.reverse_complement()
        alt = str(Seq(alt).reverse_complement())
    
    # Translate the original codon
    original_amino_acid = translate_codon(codon_seq)
    
    # Create the mutated codon
    mutation_index = (position - codon_start) % 3
    if strand == '-':
        if mutation_index == 0:
            mutation_index = 2
        elif mutation_index == 2:
            mutation_index = 0

    mutated_codon = list(codon_seq)
    mutated_codon[mutation_index] = alt
    mutated_codon = Seq(''.join(mutated_codon))
    
    # Translate the mutated codon
    mutated_amino_acid = translate_codon(mutated_codon)
    
    ref_nucl = genome[position-1]
    pos = mutation_index+1
    
    return original_amino_acid, mutated_amino_acid, codon_seq, mutated_codon, ref_nucl, pos

In [17]:
genome = load_genome("data/MPOX_genome/NC_063383.1.fasta")
gtf = load_gtf("data/MPOX_genome/GCF_014621545.1_ASM1462154v1_genomic.220824.gff")
chromosome = "NC_063383.1"

original_aa_list = []
mutated_aa_list = []
original_codon_list = []
mutated_codon_list = []
ref_nucl_list = []
nucl_pos_list = []
mutation_category = []


for row in range(0, len(article_df)):
    position = article_df.loc[row, "site"] 
    ref_nucl = article_df.loc[row, "REF"]
    alt_nucl = article_df.loc[row, "ALT"]

    node = article_df.loc[row, "parent"]
    if node == "Node1":
        original_aa, mutated_aa, original_codon, mutated_codon, ref, nucl_pos = detect_mutation_impact(genome, gtf, chromosome, position, ref_nucl, alt_nucl)
    else:  
        new_genome = find_parent_mutation(genome, node)
        original_aa, mutated_aa, original_codon, mutated_codon, ref, nucl_pos = detect_mutation_impact(new_genome, gtf, chromosome, position, ref_nucl, alt_nucl)
    
    # CDS empty
    if original_aa == "-":
        mutation_category.append("intergenic")
        original_aa_list.append('-')
        mutated_aa_list.append('-')
        original_codon_list.append('-')
        mutated_codon_list.append('-')
        nucl_pos_list.append('-')
        ref_nucl_list.append(ref)
    else:
        original_aa_list.append(original_aa)
        mutated_aa_list.append(mutated_aa)
        original_codon_list.append(str(original_codon))
        mutated_codon_list.append(str(mutated_codon))
        ref_nucl_list.append(ref)
        nucl_pos_list.append(nucl_pos)

        if mutated_aa == original_aa:
            mutation_category.append("synonymous")
        elif mutated_aa == "*":
            mutation_category.append("nonsense")
        else:
            mutation_category.append("nonsynonymous")
            
article_df["REF_v2"] = ref_nucl_list
article_df["nucl_pos_codon"] = nucl_pos_list
article_df["parent_codon_v2"] = original_codon_list
article_df["parent_aa_v2"] = original_aa_list
article_df["child_codon_v2"] = mutated_codon_list
article_df["mutated_aa_v2"] = mutated_aa_list
article_df["mutation_category_v2"] = mutation_category
print(len(article_df))
article_df.head()

800


,site,gene,direction,REF,ALT,dimer,apobec,aa_position,parent,parent_codon,...,prediction,homoplasy,occurrence,REF_v2,nucl_pos_codon,parent_codon_v2,parent_aa_v2,child_codon_v2,mutated_aa_v2,mutation_category_v2
0,3111,OPG003_CDS_176,reverse,G,A,GA,True,2.0,Node1,TCC,...,radical,False,1,G,3,ATC,I,ATT,I,synonymous
1,39139,OPG057_CDS_142,reverse,C,T,TC,True,3.0,Node1,GCG,...,NaN,False,1,C,1,GAG,E,AAG,K,nonsynonymous
2,73239,OPG093_CDS_107,forward,G,A,GA,True,2.0,Node1,AGA,...,conservative,False,1,G,1,GAT,D,AAT,N,nonsynonymous
3,74205,OPG094_CDS_106,forward,G,A,GA,True,1.0,Node1,GAA,...,moderately conservative,False,1,G,3,ATG,M,ATA,I,nonsynonymous
4,77383,OPG098_CDS_102,forward,G,A,GA,True,2.0,Node1,TGA,...,NaN,False,1,G,1,GAG,E,AAG,K,nonsynonymous


In [18]:
new_col = []
for row in range(0, len(article_df)):
    if article_df.loc[row, "REF"] != article_df.loc[row, "REF_v2"]:
        new_col.append('-')
    else:
        new_col.append("+")
article_df["new_col"] = new_col
print('does not match to reference: ', len(article_df[article_df['new_col']=='-']))
article_df[article_df['new_col']=='-']

does not match to reference:  2


,site,gene,direction,REF,ALT,dimer,apobec,aa_position,parent,parent_codon,...,homoplasy,occurrence,REF_v2,nucl_pos_codon,parent_codon_v2,parent_aa_v2,child_codon_v2,mutated_aa_v2,mutation_category_v2,new_col
268,168120,OPG193_CDS_17,forward,T,C,NaN,False,2.0,Node386,TTT,...,False,1,C,1,CTT,L,CTT,L,synonymous,-
423,81977,OPG105_CDS_95,forward,G,A,GA,True,1.0,Node561,GAC,...,False,1,A,3,CAA,Q,CAA,Q,synonymous,-


In [19]:
#filter only APOBEC mutations
df1_new1 = article_df[(article_df['REF_v2']=="G") & (article_df['ALT']=="A")]
df1_new2 = article_df[(article_df['REF_v2']=="C") & (article_df['ALT']=="T")]
df1_new = pd.concat([df1_new1, df1_new2])
df1_new = df1_new.reset_index(drop=True)

In [20]:
#filter TC and GA motifs
def load_genome(fasta_file):
    """Load the genome from a FASTA file."""
    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    new_genome = genome["NC_063383.1"].seq
    return new_genome

def find_parent_mutation(genome, parent, table=article_df):
    parent_filter_df = table[table['child']==parent]
    parent_filter_df = parent_filter_df.reset_index(drop=True)
    new_parent = parent_filter_df.loc[0, "parent"]
    sites_list = parent_filter_df.site.tolist()
    
    if new_parent == "Node1":
        new_genome = list(genome)
        for row in range(0, len(parent_filter_df)):
            position = parent_filter_df.loc[row, "site"]
            alt_nucl = parent_filter_df.loc[row, "ALT"]
            new_genome[position-1] = alt_nucl
        return Seq(''.join(new_genome))
    else:
        new_genome = list(genome)
        for row in range(0, len(parent_filter_df)):
            position = parent_filter_df.loc[row, "site"]
            alt_nucl = parent_filter_df.loc[row, "ALT"]
            new_genome[position-1] = alt_nucl
        return find_parent_mutation(new_genome, new_parent)

In [22]:
genome = load_genome("data/MPOX_genome/NC_063383.1.fasta")
motifs = []
for row in range(0, len(df1_new)):
    pos = df1_new.loc[row, 'site']
    ref = df1_new.loc[row, 'REF']
    node = df1_new.loc[row, 'parent']
    if node != "Node1":
        new_genome = find_parent_mutation(genome, node)
        if ref == "C":
            motifs.append(new_genome[pos-2:pos])
        else:
            motifs.append(new_genome[pos-1:pos+1])
    else:
        if ref == "C":
            motifs.append(genome[pos-2:pos])
        else:
            motifs.append(genome[pos-1:pos+1])

df1_new['motif'] = motifs

df1_new1 = df1_new[df1_new['motif'] == 'GA']
df1_new2 = df1_new[df1_new['motif'] == 'TC']
df1_new = pd.concat([df1_new1, df1_new2])
print(len(df1_new))

641


In [23]:
#filter Node1
df1_new = df1_new[df1_new['parent'] != 'Node1']
df1_new = df1_new.reset_index(drop=True)
len(df1_new)

631

Searching for APOBEC3 targets in MPOX genome

In [24]:
import re

## genome file
genome_file = open("data/MPOX_genome/NC_063383.1.fasta")
genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

## find coordinates of potential targets with motif in sample
## output: list with potential targets in the genome for sample
def find_potential_targets(signature='TC', genome=str(genome_dict['NC_063383.1'].seq)):
    signature_complement = str(Seq(signature).reverse_complement())
    
    #genome start with 0, write position for indexing from 1 
    targets_coordinates_TC = [i.start()+2 for i in re.finditer(signature, genome, flags=0)]
    targets_coordinates_GA = [i.start()+1 for i in re.finditer(signature_complement, genome, flags=0)]
    REF = ['C']*len(targets_coordinates_TC) + ['G']*len(targets_coordinates_GA)
    ALT = ['T']*len(targets_coordinates_TC) + ['A']*len(targets_coordinates_GA)
    
    data = {'#CHROM': ["NC_063383.1"]*len(REF),
        'position': targets_coordinates_TC+targets_coordinates_GA, 
        "REF": REF,
        "ALT": ALT}
        
    df = pd.DataFrame(data)
    return df



## find amino acids
def load_genome(fasta_file):
    """Load the genome from a FASTA file."""
    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    return genome

def load_gtf(gtf_file):
    """Load the GTF file into a pandas DataFrame."""
    gtf_columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
    gtf = pd.read_csv(gtf_file, sep="\t", comment='#', header=None, names=gtf_columns)
    return gtf

def get_cds(gtf, chromosome, position):
    """Find the CDS where the mutation is located."""
    cds_rows = gtf[(gtf['seqname'] == chromosome) & 
                   (gtf['feature'] == 'CDS') & 
                   (gtf['start'] <= position) & 
                   (gtf['end'] >= position)]
    return cds_rows

def translate_codon(codon):
    """Translate a codon to its corresponding amino acid."""
    return str(Seq(codon).translate(table=CodonTable.unambiguous_dna_by_id[11]))

def detect_mutation_impact(genome, gtf, chromosome, position, ref_allele, alt_allele):
    """Detect the amino acid change caused by a mutation."""
    cds = get_cds(gtf, chromosome, position)
    
    if cds.empty:
        return "No CDS found at this position."
    
    strand = cds.iloc[0]['strand']
        
    gene_start = cds.iloc[0]['start']
    
    # Calculate the codon position
    codon_start = gene_start + ((position - gene_start) // 3) * 3
    
    # Extract the codon
    codon_seq = genome[chromosome].seq[codon_start-1:codon_start+2]
    if strand == '-':
        codon_seq = codon_seq.reverse_complement()
        alt_allele = str(Seq(alt_allele).reverse_complement())
    
    # Translate the original codon
    original_amino_acid = translate_codon(codon_seq)
    
    # Create the mutated codon
    mutation_index = (position - codon_start) % 3
    if strand == '-':
        if mutation_index == 0:
            mutation_index = 2
        elif mutation_index == 2:
            mutation_index = 0

    mutated_codon = list(codon_seq)
    mutated_codon[mutation_index] = alt_allele
    mutated_codon = Seq(''.join(mutated_codon))
    
    # Translate the mutated codon
    mutated_amino_acid = translate_codon(mutated_codon)
    
    
    return original_amino_acid, mutated_amino_acid, codon_seq, mutated_codon, mutation_index+1

In [25]:
## create dataframe with potential APOBEC targets
df_targets = find_potential_targets()

## find amino acids
genome = load_genome("data/MPOX_genome/NC_063383.1.fasta")
gtf = load_gtf("data/MPOX_genome/GCF_014621545.1_ASM1462154v1_genomic.220824.gff")
chromosome = "NC_063383.1"


original_aa_list = []
mutated_aa_list = []
original_codon_list = []
mutated_codon_list = []
nucl_pos_list = []
mutation_category = []
for row in range(0, len(df_targets)):
    position = df_targets.loc[row, "position"]
    ref_allele = df_targets.loc[row, "REF"]
    alt_allele = df_targets.loc[row, "ALT"]
    
    try:
        original_aa, mutated_aa, original_codon, mutated_codon, nucl_pos = detect_mutation_impact(genome, gtf, chromosome, position, ref_allele, alt_allele)
    except:
        mutation_category.append("intergenic")
        original_aa_list.append('-')
        mutated_aa_list.append('-')
        original_codon_list.append('-')
        mutated_codon_list.append('-')
        nucl_pos_list.append('-')
    else:
        original_aa_list.append(original_aa)
        mutated_aa_list.append(mutated_aa)
        original_codon_list.append(original_codon)
        mutated_codon_list.append(mutated_codon)
        nucl_pos_list.append(nucl_pos)
        if mutated_aa == original_aa:
            mutation_category.append("synonymous")
        elif mutated_aa == "*":
            mutation_category.append("nonsense")
        else:
            mutation_category.append("nonsynonymous")


df_targets["nucl_pos_codon"] = nucl_pos_list
df_targets["parent_codon"] = original_codon_list
df_targets["parent_aa"] = original_aa_list
df_targets["child_codon"] = mutated_codon_list
df_targets["mutated_aa"] = mutated_aa_list
df_targets["mutation_category"] = mutation_category
df_targets.head()

,#CHROM,position,REF,ALT,nucl_pos_codon,parent_codon,parent_aa,child_codon,mutated_aa,mutation_category
0,NC_063383.1,23,C,T,-,-,-,-,-,intergenic
1,NC_063383.1,60,C,T,-,-,-,-,-,intergenic
2,NC_063383.1,78,C,T,-,-,-,-,-,intergenic
3,NC_063383.1,82,C,T,-,-,-,-,-,intergenic
4,NC_063383.1,98,C,T,-,-,-,-,-,intergenic


Saving tables

In [26]:
df_targets.to_csv("data/APOBEC_targets_aa.csv", sep="\t", index=False)

In [27]:
df1_new['#CHROM'] = "NC_063383.1"
df1_new = df1_new[['#CHROM', 'site', 'REF_v2', 'ALT', 'nucl_pos_codon', 'parent_codon_v2', 'parent_aa_v2','child_codon_v2','mutated_aa_v2','mutation_category_v2']]
df1_new.head()

,#CHROM,site,REF_v2,ALT,nucl_pos_codon,parent_codon_v2,parent_aa_v2,child_codon_v2,mutated_aa_v2,mutation_category_v2
0,NC_063383.1,186165,G,A,1,GAT,D,AAT,N,nonsynonymous
1,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
2,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
3,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
4,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous


In [28]:
df1_new.columns = df_targets.columns
df1_new.head()

,#CHROM,position,REF,ALT,nucl_pos_codon,parent_codon,parent_aa,child_codon,mutated_aa,mutation_category
0,NC_063383.1,186165,G,A,1,GAT,D,AAT,N,nonsynonymous
1,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
2,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
3,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous
4,NC_063383.1,55133,G,A,3,CTC,L,CTT,L,synonymous


In [29]:
df1_new.to_csv("data/article_pos_aa_edited.csv", sep='\t', index=False)